In [ ]:
import pandas as pd

from plotly import graph_objects as go
from plotly.offline import plot

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import tqdm

import seaborn as sns

from numba import jit

from scipy import interpolate

import shapely
from shapely.geometry import LineString, Point
import os

os.chdir(r"C:\Git\probability_calculations")

plt.rcParams["figure.figsize"] = (15,10)

import json

import seaborn as sns
import glob, os

VBA_VERSION = '7.28'
import well_model

In [ ]:
import UniflocVBA.v7_25.python_api as python_api_7_25
api = python_api_7_25.API("UniflocVBA/v7_25/UniflocVBA_7.xlam")

import UniflocVBA.v7_28.python_api as python_api_7_28
api_new = python_api_7_28.API("UniflocVBA/v7_28/UniflocVBA_7_28.xlam")
api_new.encode_PVT()

# TODO

* Физика
    * Отдебажить VBA - избавиться от пропусков в распределении
    * Сделать качественные графики узлового анализа и прочих расчетов
    * Запуск одной скважины скорректированный
    
* Логика расчетов
    * Модульность
    * Последовательность
    * Сохранение всех результатов расчета через json
    
* Ускорение расчетов - потом
    * Multiprocessing
    * json настроек
    * модульность
    
* Автоматический подбор
    * Парсинг базы БД для выбора ЭЦН для данного распределения дебита - несколько насосов + 
    * Выбор насосов + 
    * Автоматический подбор нескольких скважин
    * Поиск начального приближения для напора
    
    
* Меньше счета, больше анализа - поймать полезность / робастность / влияние на решение
    * Когда меняется решения
    * Сравнить входные распределения параметров
    * Сделать разные входные распределения
    
    
* Итоговый вывод
    * Полный pipeline по одной кнопке - исходные данные - подобрать - решение.
    * Таблица сводная с EVM по к насосам и м напорам
    
    
    
Лог    
13.05 - дебаг моделей - настройка навого pipeline

14.05 - первый расчет по новому пайплайну - определение слабых мест и мест для развития - обсуждения с РА

15.05 - дебаг физики и качественная визуализация 

16.05 - первая версия пайплайна и подбора

17.05 - анализ данных - запуск дополнительного расчета

18.05 - анализ данных - визуализация
    
19.05 - пропуск

20.05 - переход на UniflocVBA_7_27, дебаг физики

21.05 - переход на UniflocVBA_7_28
    


# Функции для работы с НРХ

In [ ]:
def plot_pump_curve(q_arr, h_esp_arr, power_esp_arr, efficiency_esp_arr, z, esp_name, f=50, fnom=50, q_work = None,
                   show = True, xlabel = None):
    q_arr = q_arr * f/fnom
    h_esp_arr = h_esp_arr * (f/fnom)**2
    power_esp_arr = power_esp_arr * (f/fnom)**2
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()

    # Offset the right spine of twin2.  The ticks and label have already been
    # placed on the right by twinx above.
    twin2.spines['right'].set_position(("axes", 1.15))

    p1, = ax.plot(q_arr, h_esp_arr, "b-",  marker = 'o', label="Напор, м")
    p2, = twin1.plot(q_arr, power_esp_arr, "r-",  marker = 'o',  label="Мощность, Вт")
    p3, = twin2.plot(q_arr, efficiency_esp_arr, "g-",  marker = 'o', label="КПД, д.ед.")

    if q_work is not None:
        f_interrr = interpolate.interp1d(q_arr,h_esp_arr, kind='cubic')
        p4 = ax.axvline(x=q_work, label=f"Рабочий режим Q={round(q_work, 2)}", linewidth=5, markersize=15)
        #p4, = ax.plot([q_work], [f_interrr(q_work)], "k",  marker = 'o', label="Рабочая точка",  markersize=15)
    
    #ax.axvspan(esp_df['Левая граница'].values[0]*f/fnom, esp_df['Правая граница'].values[0]*f/fnom, 
    #           alpha=0.2, color='green') TODO вытащить из БД
    
    if xlabel is None:
        ax.set_xlabel("Подача, м3/сут")
    else:
        ax.set_xlabel(xlabel)
    ax.set_ylabel("Напор, м")
    twin1.set_ylabel("Мощность, Вт")
    twin2.set_ylabel("КПД, д.ед.")

    ax.yaxis.label.set_color(p1.get_color())
    twin1.yaxis.label.set_color(p2.get_color())
    twin2.yaxis.label.set_color(p3.get_color())

    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
    twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
    twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
    ax.tick_params(axis='x', **tkw)
    
    if q_work is not None:
        ax.legend(handles=[p1, p2, p3, p4], loc='lower center')
    else:
        ax.legend(handles=[p1, p2, p3], loc='lower center')


    ax.grid()

    ax.set_title(f"{esp_name}") #, ступеней = {z} шт. при частоте = {f} Гц")
    if show:
        plt.show()
    else:
        additional = [p1, p2, p3, ax]
        return ax, twin1, twin2, additional
    
def interp_df(df, xname, yname, x_val, kind='linear'):
    """
    'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic',
    'previous', 'next', where 'zero', 'slinear', 'quadratic' and 'cubic
    """
    f = interpolate.interp1d( df[xname].astype(float).values,df[yname].astype(float).values, kind=kind)
    return f(x_val)


def calc_num_stages(params, api=api, api_new=api_new, vba_version = VBA_VERSION):
    if vba_version == '7.25':
        num_stages = int(params['esp_head_m'] / api.ESP_head_m(qliq_m3day=api.ESP_optRate_m3day(pump_id=params['pump_id'], freq_Hz=50),
                                        num_stages=1,
                                        freq_Hz=50,
                                        pump_id=params['pump_id'],
                                        mu_cSt=1,
                                        c_calibr=1,
                                        ))
    else:
        num_stages = int(params['esp_head_m'] / api_new.ESP_head_m(
                                                         qliq_m3day = api_new.ESP_optRate_m3day(freq_Hz=50,
                                                                                pump_id=params['pump_id'],
                                                                                mu_cSt=1,
                                                                                calibr_rate=1),
                                                        num_stages=1,
                                                        freq_Hz=50,
                                                        pump_id=params['pump_id'],
                                                        mu_cSt=1,
                                                        calibr_head=1,
                                                        calibr_rate=1,
                                                        calibr_power=1))
    return num_stages

# Функция для расчета IPR по Вогелю (быстрая) Qliq

In [ ]:
@jit(nopython=True)#, fastmath=True)
def calc_QliqVogel_m3Day(Pi,    Pr  , P_test, 
                          Wc,  pb   ):

    if Pr < pb:
        pb = Pr

    qb = Pi * (Pr - pb)
    if  Wc > 100:
        Wc = 100
    if Wc < 0:
        Wc = 0

    if (Wc == 100) or (P_test >= pb):

        calc_QliqVogel_m3Day = Pi * (Pr - P_test)

    else:
        fw = Wc / 100
        fo = 1 - fw
        qo_max = qb + (Pi * pb) / 1.8

        p_wfg = fw * (Pr - qo_max / Pi)

        if  P_test > p_wfg:
            a = 1 + (P_test - (fw * Pr)) / (0.125 * fo * pb)
            b = fw / (0.125 * fo * pb * Pi)
            c = (2 * a * b) + 80 / (qo_max - qb)
            d = (a ** 2) - (80 * qb / (qo_max - qb)) - 81
            if b == 0:
                calc_QliqVogel_m3Day = abs(d / c)
            else:
                calc_QliqVogel_m3Day = (-c + ((c * c - 4 * b * b * d) ** 0.5)) / (2 * b ** 2)

        else:
            
            CG = 0.001 * qo_max
            cd = fw * (CG / Pi) + \
              fo * 0.125 * pb * (-1 + (1 + 80 * ((0.001 * qo_max) / (qo_max - qb))) ** 0.5)
            calc_QliqVogel_m3Day = (p_wfg - P_test) / (cd / CG) + qo_max
    
    return calc_QliqVogel_m3Day

# Создание нормального распределения

In [ ]:
def create_normal_dist(mu, sigma, amount, plot=True, name='dist'):

    #mu, sigma = 100, 5 # mean and standard deviation
    
    s = np.random.normal(mu, sigma, amount)
    
    count, bins, ignored = plt.hist(s, 30, density=True)
    plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
                   np.exp( - (bins - mu)**2 / (2 * sigma**2)),
             linewidth=2, color='r')
    if plot:
        plt.title(name)
        plt.show()
        
    return s

dist = create_normal_dist(2, 1, 10**6)


#from scipy.stats import norm
## generate random numbers from N(0,1)
#data_normal = norm.rvs(size=10000,loc=0,scale=1)
#
#ax = sns.distplot(data_normal,
#                  bins=100,
#                  kde=True,
#                  color='skyblue',
#                  hist_kws={"linewidth": 15,'alpha':1})
#ax.set(xlabel='Normal Distribution', ylabel='Frequency')

# Расчетные функции

## Базовые параметры расчета

In [ ]:
params = dict(
    
    gamma_gas = 0.7, 
    gamma_oil = 0.8,
    gamma_wat = 1,
    rsb_m3m3 = 100,
    rp_m3m3 = 100,
    pb_atma = 120,
    t_res_C = 90,
    bob_m3m3 = -1,
    muob_cP = -1,
    PVTcorr = 0,
    ksep_fr = 0.6,
    
    p_bhp_atm = 70,
    
    p_wh_atm = 20,
    t_wh_c = 20,
    h_list_m = 2000,
    h_pump_m = 1800,
    diam_list_mm_casing = 150,
    diam_list_mm_tube = 73,
    
    gas_fraction_intake_d = 0.2,
    
    qliq_sm3day = 80,
    
    n_dots_for_nodal = 50,  #50 оптимум для дебага, 30 для скорости, 20 маловато - прямые линии 
    
    qliq_sm3day_range = None, #np.linspace(1, 150, 10),
    fw_perc = 20,
    hydr_corr = 1,
    temp_method = 2,
    
    freq_Hz = 53,
    
    #pump_id = 1460,
    pump_id = 2753,
    
    num_stages = 200,
    
    pi_sm3dayatm = 0.7,

    pres_atma = 180,
    
    calc_esp_new = 1,
    esp_head_m = 1400,
    ESP_gas_correct=20
)

## Расчет физической модели

### Определения функции calc_esp

In [ ]:
debug = True

def calc_esp_old(d, str_PVT_tube, p_intake, t_intake):
        # расчет эцн - переделать - нужно по ступеням
    # определение свойств ГЖС через насос
    q_mix_pump = api.MF_q_mix_rc_m3day(qliq_sm3day=d['qliq_sm3day'],
    fw_perc = d['fw_perc'],
    p_atma = p_intake,
    t_C = t_intake,
    str_PVT=str_PVT_tube)

    rho_mix_pump =  api.MF_rho_mix_kgm3(qliq_sm3day=d['qliq_sm3day'],
    fw_perc = d['fw_perc'],
    p_atma = p_intake,
    t_C = t_intake,
    str_PVT=str_PVT_tube)

    mu_mix_pump =api.MF_mu_mix_cP(qliq_sm3day=d['qliq_sm3day'],
    fw_perc = d['fw_perc'],
    p_atma = p_intake,
    t_C = t_intake,
    str_PVT=str_PVT_tube)

    mu_sst = mu_mix_pump / (rho_mix_pump/1000)

    p_esp_dis =api.ESP_head_m(qliq_m3day=q_mix_pump, num_stages=d['num_stages'],
                freq_Hz=d['freq_Hz'],
                pump_id=d['pump_id'],
                mu_cSt=mu_sst,
                c_calibr=1)*rho_mix_pump*9.81 / 10**5 + p_intake

    eff = api.ESP_eff_fr(qliq_m3day=q_mix_pump, num_stages=d['num_stages'],
                freq_Hz=d['freq_Hz'],
                pump_id=d['pump_id'],
                mu_cSt=mu_sst,
                c_calibr=1)

    head_esp = api.ESP_head_m(qliq_m3day=q_mix_pump, num_stages=d['num_stages'],
                freq_Hz=d['freq_Hz'],
                pump_id=d['pump_id'],
                mu_cSt=mu_sst,
                c_calibr=1)

    power_esp = api.ESP_power_W(qliq_m3day=q_mix_pump, num_stages=d['num_stages'],
                freq_Hz=d['freq_Hz'],
                pump_id=d['pump_id'],
                mu_cSt=mu_sst,
                c_calibr=1) / 1000
    
    gas_fraction_intake = -1
    
    return p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp


def calc_esp_new(d, str_PVT_tube, p_intake, t_intake, m_api=api):
    api = m_api
    r = api.ESP_p_atma(qliq_sm3day=d['qliq_sm3day'],
                    fw_perc=d['fw_perc'],
                    p_calc_atma=p_intake,
                    num_stages=d['num_stages'],
                    freq_Hz=d['freq_Hz'],
                    pump_id=d['pump_id'],
                    str_PVT=str_PVT_tube,
                    t_intake_C=t_intake,
                    t_dis_C=t_intake, #TODO 
                    calc_along_flow=1,
                    ESP_gas_correct=d['ESP_gas_correct'],
                    c_calibr=1,
                    dnum_stages_integrate=1,
                    out_curves_num_points=20,
                    num_value=0,
                    q_gas_sm3day=0
                  )
    r = pd.DataFrame(r)
        
    eff = r[5][0]
    head_esp = r[1][0]
    power_esp = r[8][22] / 10**3
    p_esp_dis  = r[0][0]
    t_dis = r[4][22]
    gas_fraction_intake = r[4][0]
    q_mix_pump_mean = r.iloc[3:, 6].mean()
    return p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean

#### ЭЦН UniflocVBA 7.25 тест

In [ ]:
p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean = calc_esp_new(params, api.PVT_encode_string(),
                                                                                            50, 110, m_api=api)
r

#### ЭЦН UniflocVBA 7.28 тест

In [ ]:
def calc_esp_new_7_28(params, str_PVT_tube, p_intake, t_intake, m_api=api_new):
    api_new = m_api
    # флюид
    encoded_fluid = api_new.encode_PVT(gamma_gas=params['gamma_gas'],
                                    gamma_oil=params['gamma_oil'],
                                    gamma_wat=params['gamma_wat'],
                                    rsb_m3m3=params['rsb_m3m3'],
                                    pb_atma=params['pb_atma'],
                                    t_res_C=params['t_res_C'],
                                    bob_m3m3=params['bob_m3m3'],
                                    muob_cP=params['muob_cP'],
                                    PVT_corr_set=params['PVTcorr'])
    # поток вместе с флюидом
    encoded_feed = api_new.encode_feed(q_liq_sm3day=params['qliq_sm3day'],
                                        fw_perc=params['fw_perc'],
                                        rp_m3m3=params['rp_m3m3'],
                                        q_gas_free_sm3day=-1,
                                        fluid=encoded_fluid
                                      )

    # модификация флюида
    encoded_feed_mode = api_new.feed_mod_separate_gas(k_sep = params['ksep_fr'],
                                                      p_atma = p_intake,
                                                      t_C = t_intake, 
                                                      feed = encoded_feed, 
                                                      param=''
                                                 )
    # настройки вывода ЭЦН
    param = json.dumps({'show_array': 1})
    # расчет ЭЦН по UniflocVBA 7.28
    r = api_new.ESP_p_atma(
                        p_calc_atma = p_intake,
                        t_intake_C=t_intake,
                        t_dis_C=-1, #TODO
                        feed=encoded_feed_mode,
                        pump_id=params['pump_id'],
                        num_stages=params['num_stages'],
                        freq_Hz=params['freq_Hz'],
                        calc_along_flow=True,
                        calibr_head=1,
                        calibr_rate=1,
                        calibr_power=1,
                        gas_correct_model=params['ESP_gas_correct'],
                        gas_correct_stage_by_stage=1,
                        param=param,
                        )
    
    r = pd.DataFrame(r)
        
    eff = r[6][0]
    head_esp = r[5][0]
    power_esp = r[8][0] / 10**3
    p_esp_dis  = r[0][0]
    t_dis = r[4][0]
    gas_fraction_intake = r[4][3]
    q_mix_pump_mean = r.iloc[3:, 5].mean()
    
    return p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean

In [ ]:
p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean = calc_esp_new_7_28(params, api.PVT_encode_string(), 
                                                                                                 50, 110, m_api=api_new)
r    

### Определение функции для построения НРХ в условиях работы

In [ ]:
def plot_pump_curves(params, head_esp, power_esp, eff, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean,
                     qliq_on_surface=True, vba_version = VBA_VERSION, api=api, api_new=api_new):
    to_curve_params = params.copy()
    lqliq = to_curve_params['qliq_sm3day_range']
    lp_esp_dis, lgas_fraction_intake, leff, lhead_esp, lpower_esp, lq_mix_pump_mean  = [], [], [], [], [], []
    for i in lqliq:
        to_curve_params['qliq_sm3day'] =  i
        
        if vba_version != '7.28':
            m_api = api
            ip_esp_dis, igas_fraction_intake, ieff, ihead_esp, ipower_esp, _, iq_mix_pump_mean = calc_esp_new(to_curve_params, 
                                                                                                              str_PVT_tube, p_intake, t_intake,
                                                                                                             m_api = m_api)
        else:
            m_api = api_new
            ip_esp_dis, igas_fraction_intake, ieff, ihead_esp, ipower_esp, _, iq_mix_pump_mean = calc_esp_new_7_28(to_curve_params, 
                                                                                                              str_PVT_tube, p_intake, t_intake,
                                                                                                                  m_api = m_api)
        
            
        lp_esp_dis.append(ip_esp_dis)
        lgas_fraction_intake.append(igas_fraction_intake) 
        leff.append(ieff) 
        lhead_esp.append(ihead_esp)
        lpower_esp.append(ipower_esp)
        lq_mix_pump_mean.append(iq_mix_pump_mean)
        
    if qliq_on_surface:
        q_liq_for_regime = params['qliq_sm3day']
        xlabel = 'Дебит жидкости в поверхностных условиях, м3/сут'
    else:
        q_liq_for_regime = q_mix_pump_mean
        lqliq = lq_mix_pump_mean 
        xlabel = 'Средний расход ГЖС через насос, м3/сут'

    ax, twin1, twin2, _ = plot_pump_curve(np.array(lqliq), np.array(lhead_esp), np.array(lpower_esp), 
                    np.array(leff), 1,
                    m_api.ESP_name(pump_id=to_curve_params['pump_id']) +
                    f" OPT Rate = {round(m_api.ESP_optRate_m3day(pump_id=to_curve_params['pump_id']), 2)}",
                                       #f=to_curve_params['freq_Hz'] ,  #меняет частоту
                                       f=50 ,
                                       show = False, xlabel=xlabel)

    #добавим точки, расчитанные по модели, на характеристику
    if head_esp is not None:
        ax.plot([q_liq_for_regime], [head_esp], 'bd', markersize = 20, label = 'Рабочий режим')
        twin1.plot([q_liq_for_regime], [power_esp], 'rd', markersize = 20, label = 'Рабочий режим')
        twin2.plot([q_liq_for_regime], [eff], 'gd', markersize = 20, label = 'Рабочий режим')

    plt.show()

#### Сравнение ЭЦН для 7.25 и 7.28

In [ ]:
p_intake, t_intake = 50, 70
params['ksep_fr'] = 0.7
params['qliq_sm3day_range'] = np.arange(1, 150, 5)
d = params.copy()
str_PVT_tube = api.PVT_encode_string(gamma_gas=d['gamma_gas'],
                gamma_oil=d['gamma_oil'],
                gamma_wat=d['gamma_wat'],
                rsb_m3m3=d['rsb_m3m3'],
                rp_m3m3=d['rp_m3m3'],
                pb_atma=d['pb_atma'],
                t_res_C=d['t_res_C'],
                bob_m3m3=d['bob_m3m3'],
                muob_cP=d['muob_cP'],
                PVTcorr=d['PVTcorr'],
                ksep_fr=d['ksep_fr'],
                p_ksep_atma=float(p_intake),
                t_ksep_C=float(t_intake),
                gas_only=False)
print(f"UniflocVBA 7.28")
plot_pump_curves(params, None, None,  None,  p_intake, t_intake, None, q_mix_pump_mean, 
                 qliq_on_surface=True, vba_version='7.28', api=api, api_new=api_new)
print(f"UniflocVBA 7.25")
plot_pump_curves(params, None, None,  None, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, 
                 qliq_on_surface=True, vba_version='7.25', api=api, api_new=api_new)

### Определение функции для построения КРД и КРТ

In [ ]:
def plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,
                    casing_pipe, tube_pipe, params):

    cas_df  = pd.DataFrame(casing_pipe)
    tube_df = pd.DataFrame(tube_pipe)
    
    
    index_t_c = cas_df.iloc[2][cas_df.iloc[2] == 't,C'].index[0]
    index_t_amb_c = cas_df.iloc[2][cas_df.iloc[2] == 't_amb, C'].index[0]
    index_p_atm = cas_df.iloc[2][cas_df.iloc[2] == 'p,atma'].index[0]
    index_h_m = cas_df.iloc[2][cas_df.iloc[2] == 'h,m'].index[0]

    
    
    fig = plt.Figure()
    plt.plot(cas_df.iloc[3:, index_p_atm].values, cas_df.iloc[3:, index_h_m].values, label = 'КРД в обсадной колонне')
    plt.plot(tube_df.iloc[3:, index_p_atm].values, tube_df.iloc[3:, index_h_m].values, label = 'КРД в НКТ')
    plt.plot([params['p_bhp_atm']], params['h_list_m'],  'o', label = f"Забойное давление {round(params['p_bhp_atm'], 2)} атм")
    plt.plot([params['pres_atma']], params['h_list_m'],  'ro', label = f"Пластовое давление {round(params['pres_atma'], 2)} атм")
    plt.plot([p_dis], params['h_pump_m'],  'o', label = f"Давление на выкиде ЭЦН по НКТ {round(p_dis, 2)}, атм")
    plt.plot([p_esp_dis], params['h_pump_m'],  'o', label = f"Давление на выкиде ЭЦН по ЭЦН {round(p_esp_dis, 2)}, атм")
    plt.plot([p_intake], params['h_pump_m'],  'o', label = f"Давление на приеме ЭЦН {round(p_intake, 2)}, атм")
    plt.plot([params['p_wh_atm']], [0],  'o', label = f"Давление буферное {round(params['p_wh_atm'], 2)}, атм")

    plt.legend()
    plt.grid()
    plt.title(f"Кривая распределения давления для режима Qж = {round(params['qliq_sm3day'], 2) } м3/сут")
    plt.xlabel('Давление, атм')
    plt.ylabel('Глубина, м')
    plt.gca().invert_yaxis()
    plt.show()

    cas_df  = pd.DataFrame(casing_pipe)
    tube_df = pd.DataFrame(tube_pipe)

    fig = plt.Figure()
    plt.plot(cas_df.iloc[3:, index_t_c].values, cas_df.iloc[3:, index_h_m].values,'o-', label = 'КРТ в обсадной колонне')
    plt.plot(cas_df.iloc[3:, index_t_amb_c].values, cas_df.iloc[3:, index_h_m].values, 'o-', label = 'КРТ окр. среды в обсадной колонне')

    plt.plot(tube_df.iloc[3:, index_t_c].values, tube_df.iloc[3:, index_h_m].values, 'o-', label = 'КРТ в НКТ')
    plt.plot(tube_df.iloc[3:, index_t_amb_c].values, tube_df.iloc[3:, index_h_m].values, 'o-', label = 'КРТ окр. среды в НКТ')
    plt.gca().invert_yaxis()
    plt.legend()
    plt.grid()
    plt.title(f"Кривая распределения температуры для режима Qж = {round(params['qliq_sm3day'], 2)} м3/сут")
    plt.xlabel('Температура, С')
    plt.ylabel('Глубина измеренная, м')
    plt.show()

### Определения функции calc_model

#### Модель UniflocVBA 7.25

In [ ]:
def calc_model(d, m_api = api):
        api = m_api
     # расчет в обсадной колонне КРД - до приема - снизу-вверх
        str_PVT_casing = api.PVT_encode_string(gamma_gas=d['gamma_gas'],
            gamma_oil=d['gamma_oil'],
            gamma_wat=d['gamma_wat'],
            rsb_m3m3=d['rsb_m3m3'],
            rp_m3m3=d['rp_m3m3'],
            pb_atma=d['pb_atma'],
            t_res_C=d['t_res_C'],
            bob_m3m3=d['bob_m3m3'],
            muob_cP=d['muob_cP'],
            PVTcorr=d['PVTcorr'],
            ksep_fr=-1,
            p_ksep_atma=-1,
            t_ksep_C=-1,
            gas_only=False)

        casing_pipe = api.MF_p_pipeline_atma(p_calc_from_atma=d['p_bhp_atm'],
            t_calc_from_C=d['t_res_C'],
            #t_val_C=d['t_wh_c'],
            t_val_C=[[0, d['t_wh_c']], [d['h_list_m'], d['t_res_C']]],
            h_list_m=[d['h_pump_m'], d['h_list_m']],
            diam_list_mm=d['diam_list_mm_casing'],
            qliq_sm3day=d['qliq_sm3day'],
            fw_perc=d['fw_perc'],
            q_gas_sm3day=0,
            str_PVT=str_PVT_casing,
            calc_flow_direction=0,
            hydr_corr=d['hydr_corr'],
            temp_method=2,
            c_calibr=1,
            roughness_m=0.0001,
            out_curves=2,
            out_curves_num_points=20,
            num_value=0,
            znlf=False)
        
        casing_pipe_df = pd.DataFrame(casing_pipe)
        p_intake = casing_pipe[0][0]
        t_intake = casing_pipe[0][1]

        # расчет нкт сверху вниз - определение давления на выкиде насоса
        str_PVT_tube = api.PVT_encode_string(gamma_gas=d['gamma_gas'],
            gamma_oil=d['gamma_oil'],
            gamma_wat=d['gamma_wat'],
            rsb_m3m3=d['rsb_m3m3'],
            rp_m3m3=d['rp_m3m3'],
            pb_atma=d['pb_atma'],
            t_res_C=d['t_res_C'],
            bob_m3m3=d['bob_m3m3'],
            muob_cP=d['muob_cP'],
            PVTcorr=d['PVTcorr'],
            ksep_fr=d['ksep_fr'],
            p_ksep_atma=float(p_intake),
            t_ksep_C=float(t_intake),
            gas_only=False)

        tube_pipe = api.MF_p_pipeline_atma(p_calc_from_atma=d['p_wh_atm'],
            t_calc_from_C=d['t_wh_c'],
            t_val_C=t_intake,
            h_list_m=d['h_pump_m'],
            diam_list_mm=d['diam_list_mm_tube'],
            qliq_sm3day=d['qliq_sm3day'],
            fw_perc=d['fw_perc'],
            q_gas_sm3day=0,
            str_PVT=str_PVT_tube,
            calc_flow_direction=10,
            hydr_corr=d['hydr_corr'],
            temp_method=0,
            c_calibr=1,
            roughness_m=0.0001,
            out_curves=2,
            out_curves_num_points=20,
            num_value=0,
            znlf=False)

        df = pd.DataFrame(tube_pipe)
        p_dis =  tube_pipe[0][0]
        t_dis = tube_pipe[0][1]
        
        if d['calc_esp_new'] != 1:
            #print('calc_esp_old')
            p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp = calc_esp_old(d, str_PVT_tube, p_intake, t_intake)
        else:
            #print('calc_esp_new')
            p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, esp_df , q_mix_pump_mean= calc_esp_new(d, str_PVT_tube, p_intake, t_intake,
                                                                                                            m_api = api)

        return p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                    casing_pipe, tube_pipe, q_mix_pump_mean, esp_df


In [ ]:
p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                    casing_pipe, tube_pipe, q_mix_pump_mean, esp_df= calc_model(params, m_api=api)

In [ ]:
pd.DataFrame(casing_pipe)

In [ ]:
plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, 
                 power_esp, gas_fraction_intake,
                    casing_pipe, tube_pipe, params)

#### Модель UniflocVBA 7.28

In [ ]:
def calc_model_new_7_28(d, m_api=api_new):
    ########### ЭЦН
    # флюид
    api_new = m_api
    encoded_fluid = api_new.encode_PVT(gamma_gas=d['gamma_gas'],
                                    gamma_oil=d['gamma_oil'],
                                    gamma_wat=d['gamma_wat'],
                                    rsb_m3m3=d['rsb_m3m3'],
                                    pb_atma=d['pb_atma'],
                                    t_res_C=d['t_res_C'],
                                    bob_m3m3=d['bob_m3m3'],
                                    muob_cP=d['muob_cP'],
                                    PVT_corr_set=d['PVTcorr'])

    # поток вместе с флюидом
    encoded_feed = api_new.encode_feed(q_liq_sm3day=d['qliq_sm3day'],
                                        fw_perc=d['fw_perc'],
                                        rp_m3m3=d['rp_m3m3'],
                                        q_gas_free_sm3day=-1,
                                        fluid=encoded_fluid
                                      )

    #температура
    encoded_t_model = api_new.encode_t_model(t_model=2,
                    t_list_C=[
                              [0, d['t_wh_c']],
                              [d['h_list_m'], d['t_res_C']]
                             ],
                    t_start_C=d['t_res_C'],
                    t_end_C=d['t_wh_c'],
                    param=''
                    )

    #конструкция
    encoded_pipe = api_new.encode_pipe(h_list_m=[d['h_pump_m'], d['h_list_m']], 
                                       diam_list_mm=d['diam_list_mm_casing'],
                                       roughness_m=0.0001)


    # настройки вывода ЭЦН
    param = json.dumps({'show_array': 1})

    # обсадная колонна

    casing_pipe = api_new.MF_pipe_p_atma(
         p_calc_from_atma = d['p_bhp_atm'],
        t_calc_from_C = d['t_res_C'],
        construction=encoded_pipe,
        feed=encoded_feed,
        t_model=encoded_t_model,
        calc_along_coord=False,
        flow_along_coord=False,
        flow_correlation=d['hydr_corr'],
        calibr_grav=1,
        calibr_fric=1,
        param=param, #TODO добавить в вывод gasfraction

    )

    casing_df = pd.DataFrame(casing_pipe)

    p_intake = casing_df[0][0]
    t_intake = casing_df[2][0]
    
    ########### НКТ
    # модификация флюида для НКТ
    encoded_feed_mode = api_new.feed_mod_separate_gas(k_sep = d['ksep_fr'],
                                                      p_atma = p_intake,
                                                      t_C = t_intake, 
                                                      feed = encoded_feed, 
                                                      param=''
                                                 )

    #расчет НКТ

    str_PVT_tube = None # для универсальности - в 7.25 отдельная строка

    #температура
    encoded_t_model = api_new.encode_t_model(t_model=0,
                    t_list_C=[[0, d['t_wh_c']],
                      [d['h_pump_m'], t_intake]
                     ],
                    t_start_C=d['t_wh_c'],
                    t_end_C=t_intake,
                    param=''
                    )

    #конструкция
    encoded_pipe = api_new.encode_pipe(h_list_m=[0, d['h_pump_m']], 
                                       diam_list_mm=d['diam_list_mm_tube'],
                                       roughness_m=0.001)


    tube_pipe = api_new.MF_pipe_p_atma(
                                     p_calc_from_atma = d['p_wh_atm'],
                                    t_calc_from_C = d['t_wh_c'],
                                    construction=encoded_pipe,
                                    feed=encoded_feed_mode,
                                    t_model=encoded_t_model,
                                    calc_along_coord=True,
                                    flow_along_coord=False,
                                    flow_correlation=d['hydr_corr'],
                                    calibr_grav=1,
                                    calibr_fric=1,
                                    param=param, #TODO добавить в вывод gasfraction
                                    )


    df = pd.DataFrame(tube_pipe)
    p_dis =  tube_pipe[0][0]
    t_dis = tube_pipe[0][4]
    
    p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, esp_df , \
        q_mix_pump_mean= calc_esp_new_7_28(d, str_PVT_tube, p_intake, t_intake, m_api=api_new)

    return p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                casing_pipe, tube_pipe, q_mix_pump_mean, esp_df




In [ ]:
p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                    casing_pipe, tube_pipe, q_mix_pump_mean, esp_df= calc_model_new_7_28(params, m_api=api_new)

In [ ]:
plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,
                    casing_pipe, tube_pipe, params)

### Функции для построения НРХ в условиях работы

In [ ]:
params['qliq_sm3day_range'] = range(1, 100)
plot_pump_curves(params, head_esp, power_esp, eff, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, qliq_on_surface=False)
plot_pump_curves(params, head_esp, power_esp, eff, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, qliq_on_surface=True)

## Определение основной расчетной функции узлового анализа

In [ ]:
def calc_all(d, debug=True, vba_version = VBA_VERSION, api=api, api_new=api_new):
        print('\n')
        
        if vba_version == '7.28':
            print('UniflocVBA 7.28')
            f_ipr_qliq_sm3day = api_new.IPR_q_liq_sm3day
            f_ipr_pwf_atma = api_new.IPR_p_wf_atma
            f_calc_model = calc_model_new_7_28
            m_api = api_new
            
        else:
            print('UniflocVBA 7.25')
            f_ipr_qliq_sm3day = api.IPR_qliq_sm3day
            f_ipr_pwf_atma = api.IPR_pwf_atma
            f_calc_model = calc_model
            m_api = api
            
        q_max = f_ipr_qliq_sm3day(d['pi_sm3dayatm'],
                            d['pres_atma'],
                             0,
                            d['fw_perc'],
                            d['pb_atma']) -1

        d['qliq_sm3day_range'] = list(np.linspace(1, q_max*0.2, int(d['n_dots_for_nodal']/3))) + \
                                 list(np.linspace(q_max*0.21, q_max*0.7, int(d['n_dots_for_nodal']))) + \
                                list(np.linspace(q_max*0.71, q_max, int(d['n_dots_for_nodal']*1.2)))
        #d['qliq_sm3day_range'] = np.logspace(q_max, 1, d['n_dots_for_nodal'])

        
        # создание пустых массивов для узлового анализа на выкиде насоса
        p_dis_pipe, p_dis_pump, p_intake_list, p_bhp_by_ipr_list = [], [], [], []
        
        
        # прогон по дебиту - расчет модели с разными дебитами - для решения узлового анализа - поиска рабочего режима
        pump_in_flowing_mode_count = 0
        for j, i in enumerate(d['qliq_sm3day_range']):
            print(i)
            d['qliq_sm3day'] = i

            # расчет забойного давления для данного дебита
            d['p_bhp_atm'] = f_ipr_pwf_atma(
                        d['pi_sm3dayatm'],
                         d['pres_atma'],
                        d['qliq_sm3day'],
                        d['fw_perc'],
                        d['pb_atma'],
                                )
            
            p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake, \
                        casing_pipe, tube_pipe, q_mix_pump_mean, esp_df = f_calc_model(d, m_api = m_api)
            if debug == 2:
                plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, \
                                 gas_fraction_intake, casing_pipe, tube_pipe, d)

            p_dis_pipe.append(p_dis)
            p_dis_pump.append(p_esp_dis)
            p_intake_list.append(p_intake)
            p_bhp_by_ipr_list.append(d['p_bhp_atm'])
            
            
            if p_esp_dis < p_intake:
                pump_in_flowing_mode_count += 1
                if pump_in_flowing_mode_count == 2 and j != (len(d['qliq_sm3day_range'])-1):
                    d['qliq_sm3day_range'] = d['qliq_sm3day_range'][:j+1]
                    break

        #### определение режима работы скважины с помощью узлового анализа
        
        #чистка данных от None
        df = pd.DataFrame({'qliq_sm3day_range': d['qliq_sm3day_range'],
                          'p_dis_pipe': p_dis_pipe, 
                           'p_dis_pump': p_dis_pump,
                           'p_intake': p_intake_list,
                           'p_bhp_by_ipr_list': p_bhp_by_ipr_list
                           
                          })
        df = df.dropna() 
        
        d['qliq_sm3day_range'] = df['qliq_sm3day_range'].values.flatten()
        p_dis_pipe = df['p_dis_pipe'].values.flatten()
        p_dis_pump = df['p_dis_pump'].values.flatten()
        p_intake_list = df['p_intake'].values.flatten()
        p_bhp_by_ipr_list = df['p_bhp_by_ipr_list'].values.flatten()
        
        if len(p_dis_pump) <= 1:
            print('error - all none')
            return None, None, None, None, None, None, None, 0, None, None, None, None
        
        # построение графика узлового анализа
        if debug>=1:
            fig = plt.Figure()
            plt.plot(d['qliq_sm3day_range'], p_dis_pipe, 'o-', label = 'Давление на выкиде ЭЦН, атм (НКТ)')
            plt.plot(d['qliq_sm3day_range'], p_dis_pump, 'o-', label = 'Давление на выкиде ЭЦН, атм (ЭЦН)')
            plt.plot(d['qliq_sm3day_range'], p_intake_list, 'o-', label = 'Давление на приеме ЭЦН, атм (ЭЦН)')
            plt.plot(d['qliq_sm3day_range'], p_bhp_by_ipr_list, 'o-', label = 'Давление забойное, атм')
            plt.plot(d['qliq_sm3day_range'], d['qliq_sm3day_range']*0 + d['pres_atma'], '--', label = 'Пластовое давление, атм')
            plt.plot(d['qliq_sm3day_range'], d['qliq_sm3day_range']*0 + d['pb_atma'], '--', label = 'Давление насыщения, атм')
            plt.plot(d['qliq_sm3day_range'], d['qliq_sm3day_range']*0 + d['p_wh_atm'], '--', label = 'Давление устьевое, атм')

            
            
        # решение узлового анализа - нахождение режимного дебита
        first_line = LineString(np.column_stack((d['qliq_sm3day_range'], np.array(p_dis_pipe))))
        second_line = LineString(np.column_stack((d['qliq_sm3day_range'], np.array(p_dis_pump))))
        intersection = first_line.intersection(second_line)
        
        if intersection.geom_type == 'Point':
            d['qliq_sm3day'], p_solve = float(intersection.x), float(intersection.y)
            q_liq = d['qliq_sm3day']

        else:
            print('\n no solution\n')
            head_esp, z, eff, power_esp = None, None, None, None
            return None, None, d['h_pump_m'], head_esp, z, eff, power_esp, 0, None, None, None, None
        if debug>=1:
            plt.plot([d['qliq_sm3day']], [p_solve], 'ro',   markersize=12,
                         label = f"Режимное значение дебита = {round(d['qliq_sm3day'], 2)} м3/сут")
            plt.title('Узловой анализ на выкиде ЭЦН')
            plt.ylim(0, max([d['pres_atma']] + list(p_dis_pump)) + 10 )
            plt.legend()
            plt.grid()
            plt.ylabel('Давление, атм')
            plt.xlabel('Дебит жидкости, м3/сут')
            plt.show()
        
        
        ################# фактический расчет текущего режима с получением искомых параметров
        
        #определение фактического значения дебита
        d['p_bhp_atm']  = f_ipr_pwf_atma(
                    d['pi_sm3dayatm'],
                    d['pres_atma'],
                    d['qliq_sm3day'],
                    d['fw_perc'],
                    d['pb_atma'],
                    )
        
        p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake, eff, head_esp, power_esp, gas_fraction_intake, \
                casing_pipe, tube_pipe, q_mix_pump_mean, esp_df = f_calc_model(d, m_api = m_api)
        if debug>=1:
            plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, \
                                     gas_fraction_intake, casing_pipe, tube_pipe, d)
            plot_pump_curves(d, head_esp, power_esp, eff,  p_intake, t_intake, str_PVT_tube, q_mix_pump_mean,
                            qliq_on_surface=False, vba_version=vba_version, api=api, api_new=api_new)
            plot_pump_curves(d, head_esp, power_esp, eff,  p_intake, t_intake, str_PVT_tube, q_mix_pump_mean,
                            qliq_on_surface=True, vba_version=vba_version, api=api, api_new=api_new)
        
        print('norm calculation sucessful')
        p_bhp_atm = d['p_bhp_atm']
        
        json_params = json.dumps({key: value for (key, value) in d.items() if type(value) is not np.ndarray})
        
        return casing_pipe, tube_pipe, d['h_pump_m'], head_esp, eff,\
                    power_esp, q_liq, 1, gas_fraction_intake, p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df 
    
def save_in_df(lh_mes, lhead_esp, leff, lpower_esp, lqliq, lstatus, lgas_fraction_intake, 
                   lp_bhp_atm, lp_esp_dis, lparams, num_simulations, lp_esp_dis_by_tube,
              lq_mix_pump_mean,  lesp_df,  lcasing_pipe, ltube_pipe):

    df = pd.DataFrame([lh_mes, lhead_esp, leff, lpower_esp, lqliq, lstatus, lgas_fraction_intake, 
                       lp_bhp_atm, lp_esp_dis, lparams, range(num_simulations), lp_esp_dis_by_tube,
                      lq_mix_pump_mean,  lesp_df,  lcasing_pipe, ltube_pipe]).T.dropna()


    df.columns =  ['lh_mes', 'lhead_esp', 'leff', 'lpower_esp', 
                 'lqliq', 'lstatus', 'lgas_fraction_intake', 'lp_bhp_atm',
                 'lp_esp_dis', 'lparams', 'simnumber', 'lp_esp_dis_by_tube', 'lq_mix_pump_mean',  'lesp_df',
                   'lcasing_pipe', 'ltube_pipe']
    
    df = df.sort_values(by = 'leff')
    return df

# Проверка рабочей функции

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.25',
                                                                             api=api, api_new=api_new)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
params['esp_head_m'] = 1800
params['num_stages'] = calc_num_stages(params, api=api, api_new=api_new, vba_version=VBA_VERSION)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
#params['calc_esp_new'] = 0
#casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
#p_bhp_atm, p_dis, p_esp_dis = calc_all(params)

## Анализ системы (анализ чувствительности для узлового анализа)

In [ ]:
sens_params = params.copy()
sens_params

### Коэффициент продуктивности

In [ ]:
res_list2 = []
sens_params['n_dots_for_nodal']  = 20
sens_params['pump_id'] = 2753 #100 проблемная
sens_params['esp_head_m'] = 1500
sensed_param = np.arange(0.5, 3, 0.3)
stack_values = []
for i in tqdm.tqdm(sensed_param):
    print('\n')
    print('i', i)
    sens_params['pi_sm3dayatm'] = i
    res = calc_all(sens_params, debug=1, vba_version='7.25', api=api, api_new=api_new)
    casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
    p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = res
    res_list2.append(res)
    this_values = ('head_esp',  head_esp, 'eff', eff, 'power_esp', power_esp, 'q_liq', q_liq,  'gas_fraction_intake', gas_fraction_intake,
        'p_bhp_atm', p_bhp_atm, 'p_dis', p_dis, 'p_esp_dis',  p_esp_dis, 'q_mix_pump_mean', q_mix_pump_mean)
    
    stack_values.append(this_values)
    print(this_values)

In [ ]:
fig = plt.Figure()
plt.plot([x[6] for x in res_list2],
         [x[4] for x in res_list2], 'o-', label = 'КПД, д.ед.')
plt.xlabel('Дебит жидкости в поверхностных условиях, м3/сут')
plt.ylabel('КПД, д.ед.')
plt.title('Системный анализ чувствительности')
plt.show()


for i in [
    [ [x[6] for x in res_list2], 'Дебит жидкости в поверхностных условиях, м3/сут'],
    [sensed_param, 'Кпрод, м3/сут/атм'],
    [  [x[13] for x in res_list2], 'Средний расход ГЖС через ЭЦН, м3/сут']
    ]:
    sensed_x = i[0]
    sensed_x_name = i[1]
    
    
    fig = plt.Figure()
    plt.plot(sensed_x,
             [x[11] for x in res_list2], 'o-', label = 'Давление на выкиде ЭЦН по ЭЦН, атм')
    plt.plot(sensed_x,
             [x[10] for x in res_list2], 'o-', label = 'Давление на выкиде ЭЦН по трубе, атм')
    plt.plot(sensed_x,
             [x[9] for x in res_list2], 'o-', label = 'Забойное давление, атм')
    plt.plot(sensed_x,
             [x[14][3][3] for x in res_list2], 'o-', label = 'Давление на приеме ЭЦН, атм')
    
    plt.xlabel(sensed_x_name)
    plt.ylabel('Давление, атм')
    plt.title('Системный анализ чувствительности')
    plt.legend()
    plt.show()


plt.plot(sensed_param,
         [x[4] for x in res_list2], 'o-', label = 'КПД, д.ед.')
plt.xlabel('Кпрод, м3/сут/атм')
plt.ylabel('КПД')
plt.title('Системный анализ чувствительности')
plt.legend()
plt.show()



plt.plot( [x[4] for x in res_list2],
         [x[8] for x in res_list2], 'o-', label = 'Доля газа на приеме ЭЦН (после сепарации), д.ед.')
plt.xlabel('Дебит жидкости в поверхностных условиях, м3/сут')
plt.ylabel('Доля газа на приеме ЭЦН (после сепарации), д.ед.')
plt.title('Системный анализ чувствительности')
plt.legend()
plt.show()

In [ ]:
sensed_param

In [ ]:
sens_params

In [ ]:
i_iter = -1
sens_params['bob_m3m3'] = 1.2
sens_params['muob_cP'] = 1
sens_params['diam_list_mm_tube'] = 60

sens_params['n_dots_for_nodal']  = 20
sens_params['pump_id'] = 2753 #100 проблемная
sens_params['esp_head_m'] = 1500
sens_params['pi_sm3dayatm'] = sensed_param[i_iter]
sens_params['hydr_corr'] = 1
sens_params['ksep_fr'] = 0
res = calc_all(sens_params, debug=1, vba_version='7.28', api=api, api_new=api_new)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = res

print('from loop')
print(stack_values[i_iter])
print('this')
print(( 'head_esp',  head_esp, 'eff', eff, 'power_esp', power_esp, 'q_liq', q_liq,  'gas_fraction_intake', gas_fraction_intake,
'p_bhp_atm', p_bhp_atm, 'p_dis', p_dis, 'p_esp_dis',  p_esp_dis, 'q_mix_pump_mean', q_mix_pump_mean))



### Насосы

In [ ]:
sens_params = params.copy()
sens_params['qliq_sm3day_range'] = np.arange(1, 200, 5)
for i in [2753, #страння характеристика эцн-100 - из-за округлений
          #1460, #125
           # 936, #83
          #1153, #80
         ]:

    sens_params['pump_id'] = i
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=True,
                                                                                    vba_version='7.28', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=False,
                                                                                    vba_version='7.28', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=True,
                                                                                    vba_version='7.25', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=False,
                                                                                    vba_version='7.25', api=api,
                                                                                    api_new=api_new) 
    
    

# Расчет

## Определение неопределенности работы пласта

In [ ]:
params['p_bhp_atm']

In [ ]:
debug=0
def create_q_dist(params, pi_mean = 0.8, pi_std = 0.15, 
                  pres_std = 10,
                  num_simulations= 1_000_00, debug=debug):

    all_stats_q_new =  []
    
    fig = plt.Figure()
    pi_mc = create_normal_dist(pi_mean, pi_std, 10000, name = 'Кпрод',  plot=debug)
    plt.xlabel('Коэффициент продуктивности, м3/сут/атм')
    plt.ylabel('Плотность вероятности')
    plt.title('Распределение коэффициента продуктивности')
    plt.show()
    
    
    fig = plt.Figure()
    dist_p_res =  create_normal_dist(params['pres_atma'], pres_std, 10000, name ='Pres', plot=debug)
    plt.xlabel('Пластовое давление, атм')
    plt.ylabel('Плотность вероятности')
    plt.title('Распределение пластового давления')
    plt.show()

    for i in range(num_simulations):

        new_q = calc_QliqVogel_m3Day(Pi=np.random.choice(pi_mc), 
                                            P_test =  params['p_bhp_atm'], 
                                            Pr = np.random.choice(dist_p_res), 
                                     Wc = params['fw_perc'],
                                     pb = params['pb_atma']
                            )
        all_stats_q_new.append(new_q)

    if debug>=0:
        fig = plt.Figure()

        fig, ax = plt.subplots()

        count, bins, ignored = plt.hist(all_stats_q_new, 300, [0, 400], density=True, 
                                        label=f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

        plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

        plt.title('Распределение дебита жидкости, м3/сут')
        plt.xlabel('Дебит жидкости, м3/сут')
        plt.ylabel('Плотность вероятности')
        ax.legend()
        plt.show()
    return all_stats_q_new, pi_mc, dist_p_res

all_stats_q_new, pi_mc, dist_p_res = create_q_dist(params, pi_mean = 0.8, pi_std = 0.15, 
                  pres_std = 10,
                  num_simulations= 1_000_000, debug=debug)

In [ ]:
samples = []
for i in tqdm.tqdm(range(1000)):
    val = np.random.choice(all_stats_q_new, replace=True)
    samples.append(val)
    
samples =     np.array(samples)
print('end')



In [ ]:
samples2 = []
for i in tqdm.tqdm(range(1000)):
    val = np.random.choice(all_stats_q_new, replace=True)
    samples2.append(val)
    
samples2 =     np.array(samples2)
print('end')


In [ ]:
fig, ax = plt.subplots()

ax = pd.Series(all_stats_q_new).plot.kde(label = 'Исходное распределение. ' + f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

#count, bins, ignored = plt.hist(all_stats_q_new, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

#count, bins, ignored = plt.hist(samples, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(samples), 3)} м3/сут")

ax = pd.Series(samples).plot.kde(label =  f"Выборка. Qж ср = {round(np.mean(samples), 3)} м3/сут")


plt.axvline(x=np.quantile(samples, q=0.5), c = 'r')

ax = pd.Series(samples2).plot.kde(label =  f"Выборка2. Qж ср = {round(np.mean(sasamples2mples), 3)} м3/сут")


plt.axvline(x=np.quantile(samples2, q=0.5), c = 'r')


plt.title('Распределение дебита жидкости, м3/сут')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плотность вероятности')
ax.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots()

ax = pd.Series(all_stats_q_new).plot.kde(label = 'kde_init' + f" Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

#count, bins, ignored = plt.hist(all_stats_q_new, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

count, bins, ignored = plt.hist(samples, 300, [0, 400], density=True, 
                                label=f"Qж ср = {round(np.mean(samples), 3)} м3/сут")

ax = pd.Series(samples).plot.kde(label = 'kde_sample' + f"Qж ср = {round(np.mean(samples), 3)} м3/сут")


plt.axvline(x=np.quantile(samples, q=0.5), c = 'r')


plt.title('Распределение дебита жидкости, м3/сут')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плотность вероятности')
ax.legend()
plt.show()


In [ ]:
samples

In [ ]:
samples

### Поиск подходящих насосов

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves



In [ ]:
q_median = np.quantile(all_stats_q_new, q=0.5)
q_std = np.std(all_stats_q_new)
q_p10 = np.quantile(all_stats_q_new, q=0.10)
q_p90 = np.quantile(all_stats_q_new, q=0.90)
q_std

In [ ]:
db_df_for_this_uncertainty = db_df[db_df['number_of_point_in_curves']>15]
db_df_for_this_uncertainty = db_df_for_this_uncertainty[db_df_for_this_uncertainty['rate_opt_min_sm3day']>q_p10-q_std/2]
db_df_for_this_uncertainty = db_df_for_this_uncertainty[db_df_for_this_uncertainty['rate_opt_max_sm3day']<q_p90+q_std/2]

db_df_for_this_uncertainty

In [ ]:
for i in range(db_df_for_this_uncertainty.shape[0]):
    this_pump = db_df_for_this_uncertainty.iloc[i]
    plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
        h_esp_arr = np.array(this_pump['head_points']),
        power_esp_arr = np.array(this_pump['power_points']), 
        efficiency_esp_arr = np.array(this_pump['eff_points']),
             z = 1,
                    esp_name = this_pump['name'], f=50)

In [ ]:
db_df_for_this_uncertainty['rate_nom_sm3day'].plot(kind = 'hist', bins = 100)
plt.xlabel('Номинальная подача, м3/сут')
plt.ylabel('Число ЭЦН')
plt.title('Распределение выбранных ЭЦН по подаче для текущих условий работы пласта')

In [ ]:
rates_unique = db_df_for_this_uncertainty['rate_nom_sm3day'].unique()
dq_for_group = (max(rates_unique) - min(rates_unique)) /5
rates_unique

In [ ]:
chosen_pumps = []
for i in range(5):
    q_left = min(rates_unique) + dq_for_group*i
    q_right = min(rates_unique) + dq_for_group*(i+1)
    print(q_left, q_right)
    if i == 0:
        pumps_in_group = db_df_for_this_uncertainty[(db_df_for_this_uncertainty['rate_nom_sm3day'] <= q_right) &
                                                   (db_df_for_this_uncertainty['rate_nom_sm3day'] >= q_left)]
    else:
        pumps_in_group = db_df_for_this_uncertainty[(db_df_for_this_uncertainty['rate_nom_sm3day'] <= q_right) &
                                           (db_df_for_this_uncertainty['rate_nom_sm3day'] > q_left)]
    pumps_in_group.loc[:, 'max_eff'] = pumps_in_group['eff_points'].copy().apply(lambda x: max(x))
    pumps_in_group = pumps_in_group.head(5)
    this_pumps = pumps_in_group[pumps_in_group['max_eff'] > pumps_in_group['max_eff'].max()*0.5]
    #this_pumps = pumps_in_group
    chosen_pumps.append(this_pumps)
result_pumps = pd.concat(chosen_pumps)

In [ ]:
result_pumps

In [ ]:
for i in range(result_pumps.shape[0]):
    this_pump = result_pumps.iloc[i]
    print(this_pump['rate_nom_sm3day'], this_pump['ID'])
    plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
        h_esp_arr = np.array(this_pump['head_points']),
        power_esp_arr = np.array(this_pump['power_points']), 
        efficiency_esp_arr = np.array(this_pump['eff_points']),
             z = 1,
                    esp_name = this_pump['name'], f=50)
    

## Основной цикл: Определения режима работы для выбранного(ых) насоса(ов) !!!

In [ ]:
os.chdir(r"C:\Git\probability_calculations\calc_new")

num_simulations = 3

### для массового расчета

pumps_ids = [2089, #160,
             1460, #125
            #pump_id = 1461 #200
             2753, #100
             #1868, #80
             #2289, #60
            ]
#pumps_heads = [1000, 1800] #1 запуск

pumps_heads = [1300, 1600]

# для одного расчета
pumps_ids = [1153, #80
            2753, #100 проблемная
            
            ]
pumps_heads = [1500]

#legacy
#params['pump_id'] =  936 #80 
#params['pump_id']  = 1460 #125
#params['pump_id']  = 1461 #200
#params['pump_id']  = 2799 #60   
#params['pump_id'] = 685 # 89

#params['esp_head_m'] = 1800

params['n_dots_for_nodal'] = 15
params['calc_esp_new'] = 1


def run_design(params, pumps_ids, pumps_heads, pi_mc, dist_p_res, debug=1, num_simulations=1, api=api, api_new=api_new,
              vba_version=VBA_VERSION):
    os.chdir(r"C:\Git\probability_calculations\calc_new")
    results = []
    for this_pump_id in pumps_ids:
        params['pump_id'] = this_pump_id
        print(f"pump_id = {this_pump_id}")
        for this_pump_head in pumps_heads:
            params['esp_head_m'] = this_pump_head
            print(f"esp_head_m = {this_pump_head}")

            params['num_stages'] = calc_num_stages(params, api=api, api_new=api_new, vba_version=vba_version)
                                
            lh_mes, lhead_esp,  leff, lpower_esp, lqliq,\
            lstatus, lgas_fraction_intake, lp_bhp_atm,  lp_esp_dis, lparams, lp_esp_dis_by_tube, \
                             lq_mix_pump_mean,  lesp_df , \
                    lcasing_pipe, ltube_pipe = [], [], [], \
                                            [], [], [], [], [],  [], [], [], [], [], [], []
                    
            for j, i in tqdm.tqdm(enumerate(range(num_simulations))):
            #for j, i in enumerate(range(num_simulations)):

                print('\n')
                print(f"iter = {j}")
                this_pi = np.random.choice(pi_mc)
                params['pi_sm3dayatm'] = np.random.choice(pi_mc)
                params['pres_atma'] = np.random.choice(dist_p_res)
                if this_pi <0:
                    this_pi=1

                casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake, p_bhp_atm, \
                            p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = calc_all(params, debug=debug,
                                                                                              vba_version=vba_version,
                                                                                             api=api, api_new=api_new)

                cas_df  = pd.DataFrame(casing_pipe)
                tube_df = pd.DataFrame(tube_pipe)

                lh_mes.append(h_mes)
                lhead_esp.append(head_esp)
                leff.append(eff)
                lpower_esp.append(power_esp)
                lqliq.append(q_liq)
                lstatus.append(status)
                lgas_fraction_intake.append(gas_fraction_intake)
                lp_bhp_atm.append(p_bhp_atm)
                lp_esp_dis.append(p_esp_dis)
                lparams.append(json_params)
                lp_esp_dis_by_tube.append(p_dis)
                lq_mix_pump_mean.append(q_mix_pump_mean)
                lesp_df.append(esp_df)
                lcasing_pipe.append(casing_pipe)
                ltube_pipe.append(tube_pipe)

            df = save_in_df(lh_mes, lhead_esp, leff, lpower_esp, lqliq, lstatus, lgas_fraction_intake, 
                           lp_bhp_atm, lp_esp_dis, lparams, num_simulations, lp_esp_dis_by_tube,
                            lq_mix_pump_mean,  lesp_df,  lcasing_pipe, ltube_pipe
                           )
            df.to_csv(f"res_pump_id_{this_pump_id}_head_{this_pump_head}.csv")
            results.append((this_pump_id, this_pump_head, df.copy()))

    os.chdir(r"C:\Git\probability_calculations")
    return results
                    
results = run_design(params, pumps_ids, pumps_heads, pi_mc, dist_p_res, debug=1, num_simulations=1, api=api, api_new=api_new,
              vba_version='7.25')


## Анализ результатов расчета (в т.ч. массового)

In [ ]:
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

In [ ]:
path_to_dir = r'C:\Git\probability_calculations' + '\\'
os.chdir(path_to_dir + 'calc_new_24_7_25')
os.chdir(path_to_dir + 'calc_20')

os.chdir(path_to_dir + 'calc_new')


os.chdir(path_to_dir + 'calc_22_7_25')

os.chdir(path_to_dir + 'calc_25_7_25')
#os.chdir(path_to_dir + 'calc_22_7_28')




for file in glob.glob("*.csv"):
    print(file)    
files = [x for x in glob.glob("*.csv")]


In [ ]:
dfs = []

for file_name in files:
    this_list = []
    
    file_name_splitted = file_name.split('_')
    this_id = int(file_name_splitted[-3])
    this_head = int(file_name_splitted[-1].replace('.csv', ''))
    this_name = api.ESP_name(this_id)
    this_list += [this_id, this_head, this_name]
    
    one_file = pd.read_csv(file_name, index_col = [0])
    one_file['this_id'] = this_id
    one_file['this_head'] = this_head
    one_file['this_name'] = this_name
    dfs.append(one_file.copy())

    
big_df = pd.concat(dfs)

big_df['super_id'] = big_df['this_name'].astype(str) + ' H = ' + big_df['this_head'].astype(str) + ' м'


In [ ]:

big_df

In [ ]:
pumps_in_big_df = db_df[db_df['ID'].isin(big_df['this_id'].astype(int).unique())].copy()
pumps_in_big_df.loc[:, 'max_eff'] = pumps_in_big_df['eff_points'].apply(lambda x: max(x))
pumps_in_big_df

In [ ]:
pumps_in_big_df

In [ ]:
for i in ['pi_sm3dayatm', 'p_bhp_atm', 'pres_atma']:
    big_df[i] = big_df['lparams'].apply(lambda x: json.loads(x)[i])
    

### Сводные таблицы

In [ ]:
def calc_emv(x):
    return sum(x / len(x))
def calc_std(x):
    return np.std(x)

In [ ]:
pt = big_df.pivot_table(index = ['this_name', 'this_head'], values = ['leff', 'lqliq'], aggfunc  = calc_emv)
pt

#### КПД

In [ ]:
#import matplotlib.style
#import matplotlib as mpl
#mpl.style.use('ggplot')

In [ ]:
fig = plt.Figure()

pt_eff = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['leff'], aggfunc  = calc_emv)

pt_eff_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['leff'], aggfunc  = calc_std)

pt_eff.plot(kind='bar', yerr = pt_eff_std)
plt.xlabel('ЭЦН')
plt.ylabel('КПД, %')
plt.title('Сводная анализ КПД')

plt.grid()
pt_eff

#### Дебит жидкости

In [ ]:
pt_qliq = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['lqliq'], aggfunc  = calc_emv)
pt_qliq_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['lqliq'], aggfunc  = np.std)

pt_qliq.plot(kind='bar', yerr = pt_qliq_std)
plt.xlabel('ЭЦН')
plt.ylabel('Дебит жидкости, м3/сут')
plt.title('Сводная анализ режима работы скважин')

plt.grid()
pt_qliq

#### Деградация КПД - Наработка на отказ (ННО)

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves



this_pump = db_df[db_df['ID'] == 3256]
this_pump.T

In [ ]:
def find_max_eff_by_pump_id(ID: int):
    this_pump = db_df[db_df['ID'] == ID]
    f = interpolate.interp1d(this_pump['rate_points'].values[0], this_pump['eff_points'].values[0], kind = 'quadratic')

    q_liq_range = np.arange(min(this_pump['rate_points'].values[0]), max(this_pump['rate_points'].values[0]), 1)
    #q_liq_range = this_pump['rate_points'].values[0]
    eff_interpolated = f(q_liq_range)
    fig  = plt.Figure()

    plt.plot(this_pump['rate_points'].values[0], this_pump['eff_points'].values[0], 'o-', label = 'Исходные точки с БД')
    plt.plot(q_liq_range, eff_interpolated, label = 'Проинтерполированные значения')
    plt.legend()
    plt.xlabel('Подача, м3/сут')
    plt.ylabel('КПД, д.ед.')
    plt.title(f"Pump name = {this_pump['name'].values[0]} ID = {this_pump['ID'].values[0]}")
    plt.show()
    max_init = max(this_pump['eff_points'].values[0])
    max_interpolated = max(eff_interpolated)
    print(f"max init = {max_init} max interpolated = {max_interpolated}")
    print(f"eff diff = {abs(max_init - max_interpolated)}")
    return max_init, max_interpolated

find_max_eff_by_pump_id(3259)

In [ ]:
id_max_eff_mask = {}
for i in big_df['this_id'].unique():
    _, id_max_eff_mask[i]  = find_max_eff_by_pump_id(i)
id_max_eff_mask

In [ ]:
k_degr_eff_range = np.arange(0, 1, 0.05)
def nno(x):
    return 1 - x**2

#fig = plt.Figure()
plt.plot(k_degr_eff_range, nno(k_degr_eff_range))
plt.xlabel('$1 - K_{\eta}^{дегр}$ (Деградация по КПД)')
plt.ylabel('Коэффициент снижения наработки на отказ, д.ед.')
plt.title('Условная функция вляния снижения КПД на ННО ')
plt.grid()
plt.show()


In [ ]:
big_df['this_max_eff'] = big_df['this_id'].apply(lambda x: id_max_eff_mask[x])
big_df['k_degr_eff'] = (big_df['leff']/100) / big_df['this_max_eff']
big_df['eff_div_max_eff'] = (big_df['leff']/100) / big_df['this_max_eff']

big_df['k_degr_eff'] = nno(1 - big_df['k_degr_eff'])
big_df['k_degr_eff'].plot.hist(bins=100)
plt.xlabel('k_degr_eff')

In [ ]:
pt_k_degr_eff = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['k_degr_eff'], aggfunc  = calc_emv)
pt_k_degr_eff_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['k_degr_eff'], aggfunc  = np.std)

pt_k_degr_eff.plot(kind='bar', yerr=pt_k_degr_eff_std)
pt_k_degr_eff

#### Финальная оценка

In [ ]:
pt_eff

In [ ]:
pt_eff_std

In [ ]:
def find_max_val_for_norm(pivot_table_df):
    pt_values_eff = pivot_table_df.values.flatten()
    pt_values_eff = pt_values_eff[~np.isnan(pt_values_eff)]
    pt_values_eff_max = pt_values_eff.max()
    return pt_values_eff_max

In [ ]:

pt_values_eff_max = find_max_val_for_norm(pt_eff)
pt_values_qliq_max = find_max_val_for_norm(pt_qliq)
pt_values_k_degr_eff_max = find_max_val_for_norm(pt_k_degr_eff)

pt_values_eff_std_max = find_max_val_for_norm(pt_eff_std)
pt_values_qliq_std_max = find_max_val_for_norm(pt_qliq_std)
pt_values_k_degr_eff_std_max = find_max_val_for_norm(pt_k_degr_eff_std)



print('pt_values_eff_max', pt_values_eff_max,
      'pt_values_qliq_max', pt_values_qliq_max,
      'pt_values_k_degr_eff_max', pt_values_k_degr_eff_max)

pt_result = pt_eff.copy()
pt_result = pt_eff.values/pt_values_eff_max  * pt_qliq.values/pt_values_qliq_max * pt_k_degr_eff.values/pt_values_k_degr_eff_max
pt_result_std = pt_eff_std.values/pt_values_eff_std_max  * pt_qliq_std.values / pt_values_qliq_std_max * pt_k_degr_eff_std.values / pt_values_k_degr_eff_std_max

pt_result = pd.DataFrame(pt_result, index = pt_eff.index, columns = [x[1] for x in pt_eff.columns])
pt_result_std = pd.DataFrame(pt_result_std, index = pt_eff.index, columns = [x[1] for x in pt_eff.columns])

pt_result.plot(kind='bar', yerr=pt_result_std)
plt.ylabel('Финальный рейтинг')
plt.xlabel('ЭЦН')
plt.title('Сводный рейтинг ЭЦН')

pt_result

In [ ]:
r = []
for i in pt_result.index:
    this_s = pt_result.loc[i]
    this_s.index = i + ' ' + this_s.index.astype(str)
    r.append(this_s)
pd.concat(r).sort_values()

In [ ]:
r = []
for i in pt_result.index:
    this_s = pt_result.loc[i]
    this_s.index = i + ' ' + this_s.index.astype(str)
    r.append(this_s)
pd.concat(r).sort_values(ascending=False).plot.bar()
plt.xlabel('ЭЦН')
plt.ylabel('Финальный рейтинг')

In [ ]:
pt_result_normed = pt_result / pt_result_std
pt_result_normed

In [ ]:
r = []
for i in pt_result_normed.index:
    this_s = pt_result_normed.loc[i]
    this_s.index = i + ' ' + this_s.index.astype(str)
    r.append(this_s)
pd.concat(r).sort_values(ascending=False).plot.bar()
plt.xlabel('ЭЦН')
plt.ylabel('Финальный рейтинг')
plt.title('Финальный рейтинг (комбинированный)')

## Распределения 

### КПД

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['leff'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('КПД, д.ед.')
plt.show()

### Дебита жидкости

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)].copy()

        plt.hist(this_df['lqliq'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('Дебит жидкости, м3/сут')
plt.show()

### Доли газа на приеме

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lgas_fraction_intake'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Доля газа на приеме эцн, %')
plt.xlabel('Доля газа на приеме эцн, %')
plt.show()

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lgas_fraction_intake'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


    plt.legend()
    plt.title('Доля газа на приеме эцн, %')
    plt.xlabel('Дебит жидкости, м3/сут')
    plt.show()

### Дебита ГЖС через насос

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lq_mix_pump_mean'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('Дебит ГЖС через насос, м3/сут')
plt.show()

### Распределения через KDE

In [ ]:
big_df

In [ ]:
fig, ax = plt.subplots()
big_df_t = big_df.copy()
big_df_t = big_df_t.rename(columns = {'super_id': 'ЭЦН'})
g = sns.kdeplot(data=big_df_t, x="lqliq", hue="ЭЦН", ax = ax)
plt.title('Распределения для каждого подбираемого насоса')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плоность вероятности')
#ax.legend(title='2')


In [ ]:
fig, ax = plt.subplots()
big_df_t = big_df.copy()
big_df_t = big_df_t.rename(columns = {'super_id': 'ЭЦН'})
big_df_t['leff'] = big_df_t['leff'] /100
g = sns.kdeplot(data=big_df_t, x="leff", hue="ЭЦН", ax = ax)
plt.title('Распределения для каждого подбираемого насоса')
plt.xlabel('КПД, д.ед.')
plt.ylabel('Плоность вероятности')
#ax.legend(title='2')


In [ ]:
sns.kdeplot(data=big_df, x="pi_sm3dayatm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('К прод., м3/сут/атм')

In [ ]:
sns.kdeplot(data=big_df, x="p_bhp_atm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('Забойное давление, атм')


In [ ]:
sns.kdeplot(data=big_df, x="pres_atma", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('Пластовое давление, атм')

In [ ]:
sns.kdeplot(data=big_df, x="lgas_fraction_intake", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel("Доля газа на приеме ЭЦН (1-ая ступень)")

In [ ]:
sns.kdeplot(data=big_df, x="leff", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('КПД, д.ед.')

In [ ]:
big_df

In [ ]:
sns.kdeplot(data=big_df, x="pi_sm3dayatm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('К продуктивности, м3/сут/атм')

### КПД от дебита жидкости

In [ ]:
#font = {'size'   : 1}
#matplotlib.rc('font', **font)
#plt.rcParams["font.size"] = 30

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    kind="kde",
    legend = True,
    height=15
)




In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    #kind="kde",
    legend = True,
    height=15
)


In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.displot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    #kind="kde",
    binwidth=(0.1, 0.1),
    legend = True,
    height=15
)


In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    #kind="kde",
    #binwidth=(0.1, 0.1),
    legend = True,
    height=15
)

In [ ]:
big_df_to_plot.iloc[0]

In [ ]:
big_df_to_plot

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                  'lq_mix_pump_mean': 'Средний дебит ГЖС через ЭЦН, м3/сут'
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x='Средний дебит ГЖС через ЭЦН, м3/сут', y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    kind="kde",
    legend = True,
    height=15
)

In [ ]:
fig, ax = plt.subplots()
big_df.plot.scatter(x="lqliq", y="leff", c="lpower_esp", legend=True, colormap='viridis', ax=ax)
plt.legend()
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('КПД, д.ед.')

plt.show()

### Боксплоты

In [ ]:
df_for_box = pd.DataFrame({x: big_df[big_df['super_id'] == x]['leff'] for x in big_df['super_id'].unique()}, index=range(600))
df_for_box.plot.box()
plt.xticks(rotation = 45)
plt.ylabel('КПД, д.ед.')
plt.show()

In [ ]:
df_for_box = pd.DataFrame({x: big_df[big_df['super_id'] == x]['lqliq'] for x in big_df['super_id'].unique()}, index=range(600))
df_for_box.plot.box()
plt.xticks(rotation = 45)
plt.ylabel('Дебит жидкости, д.ед.')
plt.show()

In [ ]:
big_df

### Violin plot

In [ ]:
sns.violinplot(x="super_id", y="leff", hue="this_head",
                    data=big_df,
               #palette="muted",
               #split=True
               #scale="lqliq"
              )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
sns.violinplot(x="super_id", y="lqliq", hue="this_head",
                    data=big_df,
               #width = 0.8,
               #palette="muted",
               #split=True
               #scale="lqliq"
               #scale='width'
              )
plt.xticks(rotation = 45)
plt.show()

## Один насос

In [ ]:
pumps_in_big_df

In [ ]:
this_pump

In [ ]:
this_pump = db_df[db_df['ID'] == 1460].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])

plot_pump_curve(q_arr,
    h_esp_arr,
    power_esp_arr,
    efficiency_esp_arr,
    1,
    this_pump['name'],
    )

In [ ]:
one_pump = big_df[big_df['super_id'] == big_df['super_id'].unique()[0]]
one_pump.plot.scatter(x = 'lqliq', y = 'leff')

In [ ]:
big_df.plot.scatter(x="lqliq", y="leff", c="lgas_fraction_intake", legend=True, colormap='viridis')
plt.legend()
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('КПД, д.ед.')
plt.title('КПД ЭЦН от дебита жидкости')

plt.show()

In [ ]:
big_df.plot.scatter(x="lqliq", y="lgas_fraction_intake", c="leff", colormap='viridis')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Доля газа на приеме ЭЦН, д.ед.')
plt.legend()
plt.title('Доля газа на приеме от дебита жидкости')
plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lq_mix_pump_mean", y="lgas_fraction_intake", c="leff", colormap='viridis')



plt.xlabel('Дебит жидкости, м3/сут')

plt.ylabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('Дебит жидкости от доли газа на приеме от дебита жидкости')

plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lqliq", y="lp_bhp_atm", c="leff", colormap='viridis')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Забойное давление, атм')
plt.legend()
plt.title('Доля газа на приеме от дебита жидкости')
plt.show()
#pi_sm3dayatm

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="pi_sm3dayatm", y="lqliq", c="leff", colormap='viridis')
plt.xlabel('Коэффициент продуктивности, м3/сут/атм')
plt.ylabel('Дебит жидкости, м3/сут')
plt.legend()
plt.title('Дебит жидкости от коэффициента продуктивности')
plt.show()
#pi_sm3dayatm

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="pres_atma", y="lqliq", c="leff", colormap='viridis')
plt.xlabel('Пластовое давление, атм')
plt.ylabel('Дебит жидкости, м3/сут')
plt.legend()
plt.title('Дебит жидкости от пластового давления')
plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lgas_fraction_intake", y="leff", c="lqliq", colormap='viridis')

ax.legend(list(big_df['super_id'].unique()))


plt.ylabel('КПД, д.ед.')

plt.xlabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('КПД от доли газа на приеме от дебита жидкости')


plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lgas_fraction_intake", y="lqliq", c="leff", colormap='viridis')



plt.ylabel('Дебит жидкости, м3/сут')

plt.xlabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('Дебит жидкости от доли газа на приеме от дебита жидкости')

plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=big_df, hue='super_id', x='lgas_fraction_intake', y='lqliq')
plt.legend(loc=2)
plt.savefig('scatter.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=big_df, hue='leff', x='lgas_fraction_intake', y='lqliq')
plt.legend(loc=2)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lp_esp_dis', y='lp_esp_dis_by_tube')
plt.legend(loc=2)
plt.show()
this_frac_from_big_df

In [ ]:
big_df.columns

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='lp_esp_dis')
plt.legend(loc=1)
plt.show()


In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='this_head')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='lp_bhp_atm')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lp_bhp_atm', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pres_atma', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pres_atma', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df['super_id'].unique()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df  = this_frac_from_big_df[this_frac_from_big_df['super_id'] ==this_frac_from_big_df['super_id'].unique()[0] ]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lqliq', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='error')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df.columns

In [ ]:
list(big_df['super_id'].unique())

In [ ]:
#sns.set_theme(style="ticks")
sns.pairplot(big_df[['leff', 'lqliq', 'lgas_fraction_intake', 'super_id']].copy(), hue="super_id", height=15)

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='this_max_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='k_degr_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='eff_div_max_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lgas_fraction_intake', y='eff_div_max_eff')
plt.legend(loc=3)
plt.show()

# Проверка расчета

In [ ]:
big_df['super_id'].unique()

In [ ]:
pump_super_id = '226ЭЦНАКИ5-125 H = 1600 м'
pump_super_id =  '10.1ЭЦНД5А-100 H = 1600 м'
one_pump_df_for_check_all = big_df[big_df['super_id'] == pump_super_id]
one_pump_df_for_check   = one_pump_df_for_check_all[one_pump_df_for_check_all['leff'] == one_pump_df_for_check_all['leff'].max()]
one_pump_df_for_check

In [ ]:
one_case = one_pump_df_for_check.iloc[0]
one_case

In [ ]:
params = json.loads(one_case['lparams'])
params

In [ ]:
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake, p_bhp_atm, \
                            p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = well_model.calc_all(params, debug=1,
                                                                                              vba_version='7.28',
                                                                                             api=api, api_new=api_new)

In [ ]:
this_pump = db_df[db_df['ID'] == params['pump_id']].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])
h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr
plot_pump_curve(q_arr,
    h_esp_arr,
    power_esp_arr,
    efficiency_esp_arr,
    z=1,
    esp_name= api.ESP_name(params['pump_id']),
    f=53,
    fnom=50,
    q_work=None,
    show=True,
    xlabel=None)

In [ ]:
fig, ax = plt.subplots()


ax.plot(q_arr * 53/50,efficiency_esp_arr*100, 'ro-', label ='БД НРХ' )
one_pump_df_for_check_all.plot.scatter(x = 'lq_mix_pump_mean', y = 'leff', ax =ax
                                      )

ax.legend()


In [ ]:
one_pump_df_for_check_all['lgas_fraction_intake'].plot.hist(bins=100)
plt.xlabel('Доля газа на приеме')

In [ ]:
one_pump_df_for_check_all['eff_div_max_eff'].plot.hist(bins=100)
plt.xlabel('eff_div_max_eff')

In [ ]:
one_pump_df_for_check_all

In [ ]:
api.ESP_name(params['pump_id'])

In [ ]:
api_new.ESP_name(params['pump_id'])

In [ ]:
one_case['this_name']

In [ ]:
eff

In [ ]:
one_case['leff']

## Сборка результирующего DataFrame

In [ ]:
params

In [ ]:

df = save_in_df(lh_mes, lhead_esp, leff, lpower_esp, lqliq, lstatus, lgas_fraction_intake, 
                   lp_bhp_atm, lp_esp_dis, lparams, num_simulations, lp_esp_dis_by_tube)
df

In [ ]:
debug_params = json.loads(df['lparams'][0])
debug_params['n_dots_for_nodal'] = 15
debug_params['esp_head_m'] = 1000
debug_params['num_stages'] = calc_num_stages(debug_params)
r = calc_all(debug_params)




In [ ]:
#построение графиков
#for i in df.columns:
#    #df = df.sort_values(by = i)
#    if type(df.reset_index()[i].values[0]) is not str:
#        df.reset_index()[i].plot(label = i)
#        plt.ylabel(i)
#        plt.xlabel('Порядковый номер')
#        plt.legend()
#        plt.show()

## Построение итоговых графиков 

In [ ]:
print(f"Error = {len(df['lstatus'][df['lstatus'] == 0])} из {df.shape[0]} или из {num_simulations}")
print('h_pump_m', params['h_pump_m'])
print('pump_id', params['pump_id'])
print('ESP_optRate_m3day', api.ESP_optRate_m3day(pump_id=params['pump_id']))
print('ESP_name', api.ESP_name(pump_id=params['pump_id']))



for i in [[lh_mes, 'Глубина спуска, м'],
          [lhead_esp, 'Напор, м'], 
          [leff, 'КПД, д.ед.'], 
          [lpower_esp, 'Мощность, кВт'],
          [lqliq, 'Дебит жидкости, м3/сут'],
          [lgas_fraction_intake, 'Доля газа на приеме, д.ед.'],
          [lp_bhp_atm, 'Забойное давление, атм'],
          [lp_esp_dis, 'Давление на выкиде, атм']
         ]:

    fig = plt.Figure()

    fig, ax = plt.subplots()
    
    label = i[1]
    x = np.array(i[0],dtype= np.float64)
    count, bins, ignored = plt.hist(x[~np.isnan(x)], bins=50,  density=True, label=label)
    
    
    # Расчет EMV
    leff2 = np.array(i[0], dtype = np.float64)
    leff2 = leff2[~np.isnan(leff2)]
    prob = leff2 * 0 + 1/len(leff2)

    emv = np.sum(leff2 * prob)


    plt.title(f"Распределение {label}, EMV = {round(emv, 3)}")
    plt.xlabel(label)
    ax.legend()
    plt.show()

In [ ]:
pump_id = params['pump_id']
#pump_id = 1460 #125
#pump_id = 1461 #200
#pump_id = 1868 #80
#pump_id = 2089 #160
#pump_id = 2289 #60
#pump_id = 2753 #100

qliq = np.arange(0, api.ESP_optRate_m3day(pump_id=pump_id)*2, 5)
eff = np.array([api.ESP_eff_fr(x, pump_id = pump_id, mu_cSt=1) for x in qliq])
power = np.array([api.ESP_power_W(x, pump_id = pump_id) for x in qliq])
h = np.array([api.ESP_head_m(x, pump_id = pump_id) for x in qliq])

qliq = qliq[eff >0]
power = power[eff >0]
h = h[eff >0]
eff = eff[eff >0]

plot_pump_curve(qliq, h, power, 
                eff, 1,
                api.ESP_name(pump_id=pump_id) + f" OPT Rate = {round(api.ESP_optRate_m3day(pump_id=pump_id), 2)}", f=50)

In [ ]:
plt.plot(qliq, h)

In [ ]:
plt.plot(eff/qliq, h/power)

In [ ]:
plt.plot(qliq, h/power)

In [ ]:
plt.plot(qliq,eff/qliq)

In [ ]:
plt.plot(qliq,eff, label = 'По характеристике')
plt.plot(qliq,h*1000*9.81*(qliq/86400)/power, label = 'По расчету через rho*g*h*Q/N')
plt.legend()
plt.title('Сравнение КПД прямой и косвенный расчет')
plt.ylabel('КПД, д.ед.')
plt.xlabel('Подача, м3/сут.')
plt.show()

# Анализ неоднозначности в НРХ

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

In [ ]:
db_df = pd.DataFrame(db)
db_df = db_df.T
db_df

In [ ]:
for i in db_df.columns:
    print(i)

In [ ]:
db_df.iloc[738]

In [ ]:
db_df.columns

## Анализ распределения КПД

In [ ]:
db_df[db_df['eff_max']>0]['eff_max'].plot(kind='hist', bins = 100)

## Анализ количества точек для каждой НРХ

In [ ]:
number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves

In [ ]:
db_df['number_of_point_in_curves'].plot(kind='hist', bins = 100)

In [ ]:
val = 10

db_df_filtered = db_df[db_df['number_of_point_in_curves'] <=val]
db_df_filtered = db_df_filtered[db_df_filtered['number_of_point_in_curves'] >val-1]

this_pump = db_df_filtered.iloc[2]
plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
    h_esp_arr = np.array(this_pump['head_points']),
    power_esp_arr = np.array(this_pump['power_points']), 
    efficiency_esp_arr = np.array(this_pump['eff_points']),
         z = 1,
                esp_name = this_pump['name'], f=50)

## Анализ неоднозначности НРХ

### Построение НРХ с нормированной кривой

In [ ]:
def find_amount_of_change_direction(h_esp_arr):
    if type(h_esp_arr) is list:
        h_esp_arr = np.array(h_esp_arr)
    elif type(h_esp_arr) is str:
        h_esp_arr = np.array(json.loads(h_esp_arr))

    diff = np.diff(h_esp_arr)
    change_direction = diff[1:]*diff[:-1]
    
    amount_of_change_direction = len(change_direction[change_direction<0])
    return amount_of_change_direction


clear_db_df = db_df[db_df['number_of_point_in_curves']>=10]

this_pump = clear_db_df.iloc[700]
this_pump = clear_db_df[clear_db_df['ID'] == 3256].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])
h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr



#plot_pump_curve(q_arr = q_arr, 
#    h_esp_arr = h_esp_arr,
#    power_esp_arr = power_esp_arr, 
#    efficiency_esp_arr = efficiency_esp_arr,
#         z = 1,
#    esp_name = this_pump['name'], f=50, show=True)




f=50
fnom=50
q_work=None
esp_name = this_pump['name']
z = 1

########## график
q_arr = q_arr * f/fnom
h_esp_arr = h_esp_arr * (f/fnom)**2
power_esp_arr = power_esp_arr * (f/fnom)**2
fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)

twin1 = ax.twinx()
twin2 = ax.twinx()
twin3 = ax.twinx()



# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.1))
twin3.spines['right'].set_position(("axes", 1.2))

p1, = ax.plot(q_arr, h_esp_arr, "b-",  marker = 'o', label="Напор, м")
p2, = twin1.plot(q_arr, power_esp_arr, "r-",  marker = 'o',  label="Мощность, Вт")
p3, = twin2.plot(q_arr, efficiency_esp_arr, "g-",  marker = 'o', label="КПД, д.ед.")
p4, = twin3.plot(q_arr, h_esp_arr_div_power_esp_arr, "y-",  marker = 'o', label="Напор / Мощность")


if q_work is not None:
    f_interrr = interpolate.interp1d(q_arr,h_esp_arr, kind='cubic')
    p4 = ax.axvline(x=q_work, label=f"Рабочий режим Q={round(q_work, 2)}", linewidth=5, markersize=15)
    #p4, = ax.plot([q_work], [f_interrr(q_work)], "k",  marker = 'o', label="Рабочая точка",  markersize=15)

#ax.axvspan(esp_df['Левая граница'].values[0]*f/fnom, esp_df['Правая граница'].values[0]*f/fnom, 
#           alpha=0.2, color='green') TODO вытащить из БД

ax.set_xlabel("Подача, м3/сут")
ax.set_ylabel("Напор, м")
twin1.set_ylabel("Мощность, Вт")
twin2.set_ylabel("КПД, д.ед.")
twin3.set_ylabel("Напор / Мощность")

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())
twin2.yaxis.label.set_color(p3.get_color())
twin3.yaxis.label.set_color(p4.get_color())


tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
twin3.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax.tick_params(axis='x', **tkw)

if q_work is not None:
    ax.legend(handles=[p1, p2, p3, p4], loc='lower center')
else:
    ax.legend(handles=[p1, p2, p3, p4], loc='lower center')


ax.grid()

ax.set_title(f"{esp_name}, ступеней = {z} шт. при частоте = {f} Гц")
plt.show()



find_amount_of_change_direction(h_esp_arr), find_amount_of_change_direction(power_esp_arr), find_amount_of_change_direction(efficiency_esp_arr)

### Расчет количества перегибов на кривых и нормированной кривой для каждого ЭЦН

In [ ]:
clear_db_df['h_esp_arr_div_power_esp_arr'] = None
clear_db_df['power_points_left'] = None

for i in clear_db_df.index:
    
    clear_db_df.loc[i, 'h_esp_arr_div_power_esp_arr'] = json.dumps(list( np.array(clear_db_df.loc[i, 'head_points']) /  \
                                                        np.array(clear_db_df.loc[i, 'power_points'])))
    
    
    this_pump = clear_db_df.loc[i]

    power_points = np.array(this_pump['power_points'])
    rate_points = np.array(this_pump['rate_points'])

    power_points_left = power_points[rate_points< 0.3*max(rate_points)]
    
    
    clear_db_df.loc[i, 'power_points_left'] = json.dumps(list(power_points_left))

    
    

clear_db_df.loc[:, 'amount_of_change_direction_head'] = clear_db_df['head_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_eff'] = clear_db_df['eff_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_power'] = clear_db_df['power_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_h_esp_div_power'] = clear_db_df['h_esp_arr_div_power_esp_arr'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_power_points_left'] = clear_db_df['power_points_left'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_combined_solution'] = clear_db_df.apply(lambda x: min(x['amount_of_change_direction_power_points_left'], 
                                                                x['amount_of_change_direction_h_esp_div_power']), axis=1)


In [ ]:
clear_db_df

### Построение распределений для всей БД - количество перегибов для каждой кривой

In [ ]:
clear_db_df_with_calc = clear_db_df[clear_db_df['amount_of_change_direction_eff'] == 1]

In [ ]:
fig = plt.Figure()
ax = fig.add_axes([0,0,1,1])
for j, i in enumerate(clear_db_df_with_calc.columns[-6:]):
    v = clear_db_df_with_calc[i].value_counts() / clear_db_df_with_calc.shape[0]*100
    #v = clear_db_df_with_calc[i].value_counts() 
    v = v[v.index<5]
    plt.bar(v.index.values+0.1*j, v.values, label=i, width= 0.1, align='edge')
    plt.legend()
    #plt.show()
plt.grid()
plt.ylabel('Количество насосов')
plt.xlabel('Количество точек перегиба')
plt.show()

In [ ]:
#fig, ax = plt.subplots()

fig = plt.Figure()

#ax = fig.add_axes([0,0, 1, 1])
#for i in clear_db_df.columns[-4:]:
i = clear_db_df.columns[-1]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.00, v.values, label=i, width=0.25, align='center')

i = clear_db_df.columns[-2]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.25, v.values, label=i, width=0.25,  align='center')

i = clear_db_df.columns[-4]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.5, v.values, label=i, width=0.25,  align='center')
    
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')

plt.legend()
#fig.tight_layout()

plt.show()

In [ ]:
clear_db_df[clear_db_df.columns[-1]].plot(kind='hist', bins = [0, 1, 2, 3, 4, 5], align='left', legend = 'Неопределенность по H/N')
clear_db_df[clear_db_df.columns[-2]].plot(kind='hist', bins = [0, 1, 2, 3, 4, 5], align='left', legend = 'Неопределенность по H/N')
plt.show()

### Построение конкретных кривых

In [ ]:
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_eff']==1]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_power']==2]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_head']==1]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_head']==1]

clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_h_esp_div_power']==1]
clear_db_df_with_calc_filtered = clear_db_df_with_calc_filtered.head(10)

print(clear_db_df_with_calc_filtered.shape)
for i in range(clear_db_df_with_calc_filtered.shape[0]):
    this_pump = clear_db_df_with_calc_filtered.iloc[i]


    q_arr = np.array(this_pump['rate_points'])
    h_esp_arr = np.array(this_pump['head_points'])
    power_esp_arr = np.array(this_pump['power_points']) 
    efficiency_esp_arr = np.array(this_pump['eff_points'])
    h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr



    #plot_pump_curve(q_arr = q_arr, 
    #    h_esp_arr = h_esp_arr,
    #    power_esp_arr = power_esp_arr, 
    #    efficiency_esp_arr = efficiency_esp_arr,
    #         z = 1,
    #    esp_name = this_pump['name'], f=50, show=True)

    f=50
    fnom=50
    q_work=None
    esp_name = this_pump['name']
    z = 1

    ########## график
    q_arr = q_arr * f/fnom
    h_esp_arr = h_esp_arr * (f/fnom)**2
    power_esp_arr = power_esp_arr * (f/fnom)**2
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()



    # Offset the right spine of twin2.  The ticks and label have already been
    # placed on the right by twinx above.
    twin2.spines['right'].set_position(("axes", 1.1))
    twin3.spines['right'].set_position(("axes", 1.2))

    p1, = ax.plot(q_arr, h_esp_arr, "b-",  marker = 'o', label="Напор, м")
    p2, = twin1.plot(q_arr, power_esp_arr, "r-",  marker = 'o',  label="Мощность, Вт")
    p3, = twin2.plot(q_arr, efficiency_esp_arr, "g-",  marker = 'o', label="КПД, д.ед.")
    p4, = twin3.plot(q_arr, h_esp_arr_div_power_esp_arr, "y-",  marker = 'o', label="Напор / Мощность")


    if q_work is not None:
        f_interrr = interpolate.interp1d(q_arr,h_esp_arr, kind='cubic')
        p4 = ax.axvline(x=q_work, label=f"Рабочий режим Q={round(q_work, 2)}", linewidth=5, markersize=15)
        #p4, = ax.plot([q_work], [f_interrr(q_work)], "k",  marker = 'o', label="Рабочая точка",  markersize=15)

    #ax.axvspan(esp_df['Левая граница'].values[0]*f/fnom, esp_df['Правая граница'].values[0]*f/fnom, 
    #           alpha=0.2, color='green') TODO вытащить из БД

    ax.set_xlabel("Подача, м3/сут")
    ax.set_ylabel("Напор, м")
    twin1.set_ylabel("Мощность, Вт")
    twin2.set_ylabel("КПД, д.ед.")
    twin3.set_ylabel("Напор / Мощность")

    ax.yaxis.label.set_color(p1.get_color())
    twin1.yaxis.label.set_color(p2.get_color())
    twin2.yaxis.label.set_color(p3.get_color())
    twin3.yaxis.label.set_color(p4.get_color())


    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
    twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
    twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
    twin3.tick_params(axis='y', colors=p4.get_color(), **tkw)

    ax.tick_params(axis='x', **tkw)

    if q_work is not None:
        ax.legend(handles=[p1, p2, p3, p4], loc='lower center')
    else:
        ax.legend(handles=[p1, p2, p3, p4], loc='lower center')


    ax.grid()

    ax.set_title(f"{esp_name}, ступеней = {z} шт. при частоте = {this_pump['freq_Hz']} Гц")
    plt.show()



    print(
        'h', find_amount_of_change_direction(h_esp_arr),
        'power', find_amount_of_change_direction(power_esp_arr), \
    'eff', find_amount_of_change_direction(efficiency_esp_arr), 
        'h/power', find_amount_of_change_direction(h_esp_arr_div_power_esp_arr),
        'final_solution', this_pump['amount_of_change_direction_combined_solution']
        
        
    )

In [ ]:
#график без номированной НРХ
plot_pump_curve(q_arr, h_esp_arr, power_esp_arr, efficiency_esp_arr, 1000, esp_name, f=50, fnom=50, q_work = None,
                   show = True)

### Расчет итоговой выборки

In [ ]:
clear_db_df_with_calc.shape[0], db_df.shape[0], clear_db_df.shape[0]
